<a href="https://colab.research.google.com/github/jmhuer/quantum_dots/blob/main/quantum_dots_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data from excell


In [1]:
!pip install skorch
!pip install tqdm
!git clone https://github.com/jmhuer/optimization_tools

from optimization_tools.utils import download_gdrive
data = '14mee8d0GDbwNzIprVSJoWdojGtqO_aTX' ##google drive id of excell 
download_gdrive(data)

     |████████████████████████████████| 133kB 22.8MB/s 
Cloning into 'optimization_tools'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 46 (delta 12), reused 39 (delta 8), pack-reused 0
Unpacking objects: 100% (46/46), done.
Downloading...
From: https://drive.google.com/uc?id=14mee8d0GDbwNzIprVSJoWdojGtqO_aTX
To: /content/QDots data - Juan.xlsx
100%|##########| 39.7k/39.7k [00:00<00:00, 4.10MB/s]



In [2]:
import pandas as pd

def open_excel(filename):
    excell = pd.ExcelFile(filename)
    excell.sheet_names
    df = excell.parse("Sheet1")
    df.columns = df.columns.map(str)
    df = df.dropna().reset_index(drop=True)
    return df

df = open_excel("/content/QDots data - Juan.xlsx")

In [3]:
df.head()

,Unnamed: 0,Time (min),μL,nM λ1,nM λ2,λ1 (nm),λ2 (nm),# emitters λ1 (x1010),# emitters λ2 (x1010),415,445,480,515,555,590,630,680
0,2021-04-12,0.0,0.0,0.0,0.0,620.0,560.0,0.0000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0
1,2021-04-12,0.0,30.0,50.0,0.0,620.0,560.0,90.3321,0.0,9.0,20.0,21.0,67.0,106.0,666.0,4320.0,24.0
2,2021-04-12,1.0,30.0,50.0,0.0,620.0,560.0,90.3321,0.0,9.0,21.0,22.0,69.0,107.0,675.0,4413.0,25.0
3,2021-04-12,2.0,30.0,50.0,0.0,620.0,560.0,90.3321,0.0,9.0,21.0,22.0,69.0,107.0,677.0,4439.0,25.0
4,2021-04-12,3.0,30.0,50.0,0.0,620.0,560.0,90.3321,0.0,9.0,21.0,22.0,69.0,106.0,678.0,4457.0,25.0


## Pre-processing + model defenitions before training

In [4]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR


##LINEAR REGRESSION
lr_reg = make_pipeline(LinearRegression())

##RandomForestRegressor
rf_reg = make_pipeline(StandardScaler(), RandomForestRegressor())
parameters = {'randomforestregressor__max_depth':[1,2,3,4,5]}
rf_reg = GridSearchCV(rf_reg, parameters)
  
##SVM
svr = make_pipeline(StandardScaler(), SVR(kernel='linear'))
parameters = {'svr__kernel':['linear', 'rbf'], 'svr__epsilon':[0.1, 0.2, 0.3,0.6], 'svr__C': [0.1,0.5,1,2,3]}
svr = GridSearchCV(svr, parameters)


# NN stuff here

In [5]:
from torch import nn
from skorch import NeuralNetRegressor

class MultivariateLinearRegression(nn.Module):
    def __init__(self, input = 8, output=1, num_units=10, nonlin=nn.LeakyReLU()):
          super(MultivariateLinearRegression, self).__init__()
          self.dense0 = nn.Linear(input, num_units)
          self.nonlin = nonlin
          self.dropout = nn.Dropout(0.1)
          self.output = nn.Linear(num_units, output)
    def forward(self, X):
          X = self.nonlin(self.dense0(X))
          X = self.dropout(X)
          X = self.output(X)
          return X





## single variable 
net = NeuralNetRegressor(
    MultivariateLinearRegression().double(),
    iterator_train__shuffle = False,
    train_split = False,
    verbose = 0)

myNN_reg = make_pipeline(StandardScaler(), net)

params = {
    'neuralnetregressor__lr': [0.001, 0.002, 0.003],
    'neuralnetregressor__max_epochs': [10, 20, 30]
}
myNN_reg = GridSearchCV(myNN_reg, params)




## multivariate variable 
net2 = NeuralNetRegressor(
    MultivariateLinearRegression(output=2).double(),
    iterator_train__shuffle = False,
    train_split = False,
    verbose = 0)

myNN_reg2 = make_pipeline(StandardScaler(), net2)

params = {
    'neuralnetregressor__lr': [0.001, 0.002, 0.003],
    'neuralnetregressor__max_epochs': [10, 20, 30]
}
myNN_reg2 = GridSearchCV(myNN_reg2, params)


# overfit criteria 

In [6]:
# calculate aic for regression
def calculate_aic(n, mse, num_params):
	aic = n * log(mse) + 2 * num_params
	return aic


# calculate bic for regression
def calculate_bic(n, mse, num_params):
	bic = n * log(mse) + num_params * log(n)
	return bic

##Training loop w/ cross-validation 

In [7]:
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

##function to help store history and metrics of each cross validation run
history = dict()
def append_history(history, algorithm, name, X_test, Y_test):
    #calculate performance metrics
    Y_pred = algorithm.predict(X_test)
    score = algorithm.score(X_test, Y_test)
    #if dict does not contain current alg, create a key
    if name not in history.keys(): 
        history[name] = {"mse": [], "r2": [], "predictions": [], "actual": []}
    #store preformance metrics & history
    mse = mean_squared_error(Y_test, Y_pred)
    history[name]["mse"].append(mse)
    history[name]["r2"].append(score)
    history[name]["predictions"].append(Y_pred)
    history[name]["actual"].append(Y_test)
    return history



X_columns  = ["415","445","480", "515", "555", "590", "630", "680"]
Y1_columns = ["# emitters λ1 (x1010)"]
Y2_columns = ["# emitters λ2 (x1010)"]
Y_columns  = ["# emitters λ1 (x1010)", "# emitters λ2 (x1010)"]


##For Cross validations 
n_splits = 10
kf = KFold(n_splits=n_splits, random_state=42, shuffle=True) # Define the split - into 10 folds 

##A progress bar because Im fancy
pbar = tqdm(total=100,position=0, leave=True)

##MAIN LOOP
for train_index, test_index in kf.split(df):
    X_train, X_test = df.loc[train_index, X_columns], df.loc[test_index, X_columns]
    Y_train, Y_test = df.loc[train_index, Y2_columns], df.loc[test_index, Y2_columns]
    Y_train_both, Y_test_both = df.loc[train_index, Y_columns], df.loc[test_index, Y_columns]

    ##NN Regressor multivariate
    myNN_reg2.fit(X_train, Y_train_both.values)
    history = append_history(history, myNN_reg2, "myNN_reg2", X_test, Y_test_both)
    
    ##LINEAR REGRESSION
    lr_reg.fit(X_train, Y_train.values.ravel())
    history = append_history(history, lr_reg, "linear_regression", X_test, Y_test)

    ##RandomForestRegressor
    rf_reg.fit(X_train, Y_train.values.ravel())
    history = append_history(history, rf_reg, "rf_reg", X_test, Y_test)

    ##Support Vecotor Regressor 
    svr.fit(X_train, Y_train.values.ravel())
    history = append_history(history, svr, "svr", X_test, Y_test)

    # ##NN Regressor
    myNN_reg.fit(X_train, Y_train.values)
    history = append_history(history, myNN_reg, "myNN_reg", X_test, Y_test)

    pbar.update(10)
pbar.close()

print('\n')
for i in history.keys():
    print('\n', i," : ", sum(history[i]["mse"]) / len(history[i]["mse"]))
    print("---")


# num_params = len(lr_reg.best_estimator_[1].coef_) + 1
# print(calculate_bic(len(Y_test), history["linear_regression"]["mse"], num_params))

100%|██████████| 100/100 [02:20<00:00,  1.40s/it]




 myNN_reg2  :  17.872397607929535
---

 linear_regression  :  3.533323019286695
---

 rf_reg  :  1.0549915848212859
---

 svr  :  4.083087092195877
---

 myNN_reg  :  7.034157507923827
---


In [8]:
print('\n')
for i in history.keys():
    print('\n', i," : ", sum(history[i]["r2"]) / len(history[i]["r2"]))
    print("---")




 myNN_reg2  :  0.9764709038167068
---

 linear_regression  :  0.9952320133708717
---

 rf_reg  :  0.9987279393045411
---

 svr  :  0.9946020805224178
---

 myNN_reg  :  0.9905178786637414
---


# Ploting utils

In [9]:
import plotly.graph_objects as graph
def plot(all_history:list, title:str, log = False):
    """
    input:
        all_history: list of dicts to plot
    ret:
        None: show plotly fig
    """
    symbol_sequence= ['circle-open', 'circle', 'circle-open-dot', 'square']
    fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text=title))) 
    for i in range(len(all_history)):
        fig.add_trace(graph.Scatter(x = all_history[i]["x"], 
                                    y = all_history[i]["y"],
                                    name = all_history[i]["legend"],
                                    mode='markers',
                                    marker_size=5,
                                    marker_symbol=all_history[i]["marker_symbol"])) 
    if log: fig.update_xaxes(type="log")
    fig.show()


# Plot each CV split validation on same plot predicted vs actual

In [ ]:
import numpy as np
names = list(history.keys())

n_splits = 10

#perfect plot
perfect_plot = {"legend": "actuals", 
                "x": list(np.linspace(0, 100, num=1000, endpoint=True)), 
                "y": list(np.linspace(0, 100, num=1000, endpoint=True)),
                "marker_symbol": 'line-ne-open'}
X,Y = [], []           
for alg in names:
  for i in range(n_splits):
      X = X + list(history[alg]["actual"][i].values.ravel())
      Y = Y + list(history[alg]["predictions"][i].ravel())

  current_plot = {"legend": "predictions", 
                  "x": X, 
                  "y": Y,
                  "marker_symbol": 'star'}

  plot([current_plot, perfect_plot], alg + " predicted v actual")